In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Data Security:

In [ ]:
# Short exploration with pandas
dataframe = pd.read_csv("Data_Security.csv")

# About data
print(len(dataframe["Security Measure"].unique())) # --> 10  possible unique values

#rename column for huggingface API
dataframe.rename(columns={'Security Measure': 'labels'}, inplace=True)


dataframe.head()


10


,annotationID,segmentID,category,segment,labels
0,20466,31,Data Security,Security <br> <br> This Web site takes every p...,Generic
1,20467,32,Data Security,When our registration/order form asks users to...,Secure data transfer
2,20468,32,Data Security,When our registration/order form asks users to...,Generic
3,20476,33,Data Security,While we use SSL encryption to protect sensiti...,Secure data storage
4,20477,33,Data Security,While we use SSL encryption to protect sensiti...,Privacy training


In [ ]:
# Encode labels and split data
train_df, eval_df = train_test_split(dataframe, test_size=0.2, random_state=42)

# Initialize a label encoder for each target column
encoder = LabelEncoder()

# encode training dataset
train_df['labels'] = encoder.fit_transform(train_df['labels'])

# Encode eval dataset
eval_df['labels'] = encoder.fit_transform(eval_df['labels'])

# transform to huggingface dataset
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)


In [ ]:
# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['segment'], padding="max_length", truncation=True)

# Tokenize the dataset
train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
eval_tokenized_dataset = eval_dataset.map(tokenize_function, batched=True)

# Prepare DataLoader
small_train_dataset = train_tokenized_dataset.shuffle(seed=42).select(range(200))  # Small subset for example
small_eval_dataset = eval_tokenized_dataset.shuffle(seed=42).select(range(100))



Map:   0%|          | 0/806 [00:00<?, ? examples/s]

Map:   0%|          | 0/202 [00:00<?, ? examples/s]

In [ ]:
print(small_train_dataset.column_names)

['annotationID', 'segmentID', 'category', 'segment', 'labels', '__index_level_0__', 'input_ids', 'attention_mask']


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=10)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)

    # Calculate precision, recall, and F1-score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset, # MAKE SURE YOU ARE USING CORRECT DATASET
    eval_dataset=eval_tokenized_dataset,  # MAKE SURE YOU ARE USING CORRECT DATASET
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model

eval_results = trainer.evaluate()
print(eval_results)

# Save the model
model.save_pretrained("./data_security_model")
tokenizer.save_pretrained("./data_security_model")


<ipython-input-30-cb094f8c7ced>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.818759,0.381188,0.200853,0.381188,0.261390
2,No log,1.733869,0.366337,0.320355,0.366337,0.283467
3,No log,1.708725,0.381188,0.308278,0.381188,0.320922


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.708725094795227, 'eval_accuracy': 0.3811881188118812, 'eval_precision': 0.30827777486025404, 'eval_recall': 0.3811881188118812, 'eval_f1': 0.3209223061657793, 'eval_runtime': 2.9025, 'eval_samples_per_second': 69.595, 'eval_steps_per_second': 8.958, 'epoch': 3.0}


('./data_security_model/tokenizer_config.json',
 './data_security_model/special_tokens_map.json',
 './data_security_model/vocab.txt',
 './data_security_model/added_tokens.json')

# Do Not Track

In [6]:
# Short exploration with pandas
import pandas as pd
dataframe = pd.read_csv("International_and_Specific_Audiences.csv")

# About data
print(len(dataframe["Audience Type"].unique())) # --> 10  possible unique values

#rename column for huggingface API
dataframe.rename(columns={'Audience Type': 'labels'}, inplace=True)


dataframe.head()


5


,annotationID,segmentID,category,segment,labels
0,20331,4,International and Specific Audiences,Sci-News.com does not knowingly collect or sol...,Children
1,20594,4,International and Specific Audiences,Sci-News.com does not knowingly collect or sol...,Children
2,20243,4,International and Specific Audiences,Sci-News.com does not knowingly collect or sol...,Children
3,13297,35,International and Specific Audiences,Children's Privacy <br> <br> Our Services are ...,Children
4,13296,36,International and Specific Audiences,If you have questions concerning our informati...,Children
